In [2]:
import numpy as np
import pandas as pd
from matplotlib.pyplot import subplots
from statsmodels.datasets import get_rdataset 
import sklearn.model_selection as skm
from ISLP import load_data , confusion_table 
from ISLP.models import ModelSpec as MS

In [3]:
from sklearn.tree import (DecisionTreeClassifier as DTC, DecisionTreeRegressor as DTR, plot_tree , export_text)
from sklearn.metrics import (accuracy_score ,log_loss) 
from sklearn.ensemble import \
(RandomForestRegressor as RF, GradientBoostingRegressor as GBR)
from ISLP.bart import BART

In [4]:
from sklearn.metrics import (accuracy_score,log_loss)
from sklearn.ensemble import (RandomForestRegressor as RF,GradientBoostingRegressor as GBR,
                             GradientBoostingClassifier as GBC,
                             BaggingClassifier as BC)

In [145]:
pred = pd.read_csv("/Users/christianhellum/Cand. Merc./Data-Science-Project/data_science_project/Beat the bookies/df_prepro_2.csv", index_col=0)

In [146]:
pred

,Date,Home,Away,FTR,Attendance,B365H,B365D,B365A,AttackStrengthHome,AttackStrengthAway,...,AvgAwayShotsOnTargetLast5,elo,elo_away,FormHomeTeam,FormAwayTeam,ProbabilityDraw,ProbabilityHomeWin,ProbabilityAwayWin,B365_guess,B365_right_or_wrong
0,2019-08-16,2,4,H,47693.0,5.25,3.80,1.65,0.000000,0.000000,...,0.0,1722.019409,2001.228638,0.000000,0.000000,0.226260,0.127955,0.645785,A,wrong
1,2019-08-17,6,21,A,23566.0,4.75,4.20,1.65,0.000000,0.000000,...,0.0,1681.163452,1841.848999,0.000000,0.000000,0.239320,0.266267,0.494412,A,correct
2,2019-08-17,24,22,D,41846.0,1.66,3.75,5.50,0.000000,0.000000,...,0.0,1813.655029,1719.431152,0.000000,0.000000,0.229859,0.616701,0.153439,H,wrong
3,2019-08-17,18,8,H,15127.0,2.80,3.20,2.60,0.000000,0.000000,...,0.0,1566.961060,1712.778198,0.000000,0.000000,0.240204,0.286297,0.473499,A,wrong
4,2019-08-17,16,19,A,10020.0,2.00,3.20,4.20,0.000000,0.000000,...,0.0,1670.794800,1641.649048,0.000000,0.000000,0.237143,0.533531,0.229326,H,wrong
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1855,2024-05-05,19,5,A,18300.0,3.20,3.25,2.38,0.034274,0.044503,...,4.4,1629.822266,1696.406372,2057.936660,2057.038916,0.241897,0.398349,0.359754,A,correct
1856,2024-05-05,6,26,H,19548.0,2.30,3.60,2.88,0.028226,0.065104,...,3.4,1628.082397,1724.820923,2338.240591,3718.007305,0.241856,0.355002,0.403142,H,correct
1857,2024-05-05,24,0,A,44643.0,2.10,3.00,4.00,0.038076,0.028497,...,3.4,1687.701660,1622.472412,2059.229795,1340.504648,0.233489,0.580879,0.185632,H,wrong
1858,2024-05-05,20,1,A,10864.0,1.57,4.00,5.75,0.032064,0.041344,...,2.6,1621.235840,1527.045044,2925.893320,1647.954297,0.229864,0.616662,0.153475,H,wrong


In [147]:
pred['Home'] = pred['Home'].astype('category')
pred['Away'] = pred['Away'].astype('category')
pred['B365_right_or_wrong'] = pred['B365_right_or_wrong'].astype('category')

In [148]:
model = MS(pred.columns.drop(['FTR', 'Date', 'B365_guess', 'B365_right_or_wrong']), intercept=False)
D = model.fit_transform(pred)
feature_names = list(D.columns)
X = np.asarray(D)

In [149]:
y=pred['B365_right_or_wrong']
(X_train, X_test, y_train, y_test) = skm.train_test_split(X,y,train_size=1478,random_state=1)

In [150]:
#Boosting
boost_pred = GBC(n_estimators=5000,
learning_rate=0.2, max_depth=3, random_state=0)
boost_pred.fit(X_train, y_train)

GradientBoostingClassifier(learning_rate=0.2, n_estimators=5000, random_state=0)

In [151]:
ypred = boost_pred.predict(X_test)
confusion_boost=confusion_table(y_test,ypred)
confusion_boost

Truth,correct,wrong
Predicted,,
correct,119,76
wrong,86,101


In [139]:
(11+8+59)/np.sum(np.sum(confusion_boost))

0.4875

In [152]:
print(classification_report(y_test, ypred))

              precision    recall  f1-score   support

     correct       0.58      0.61      0.59       195
       wrong       0.57      0.54      0.55       187

    accuracy                           0.58       382
   macro avg       0.58      0.58      0.57       382
weighted avg       0.58      0.58      0.58       382



In [153]:
#Bagging
bag_weekly = RF(max_features=X_train.shape[1], random_state=1)
y_train=np.where(y_train=='correct',1,0)
bag_weekly.fit(X_train, y_train)

RandomForestRegressor(max_features=83, random_state=1)

In [154]:
ypred = bag_weekly.predict(X_test)
y_pred=np.where(ypred>0.5,'correct','wrong')
confusion_bagging=confusion_table(y_test,y_pred)

In [155]:
confusion_bagging

Truth,correct,wrong
Predicted,,
correct,114,81
wrong,85,102


In [65]:
pred=pd.DataFrame({'importance':bag_weekly.feature_importances_},index=feature_names)
pred.sort_values(by='importance', ascending=False)

,importance
B365D,0.075093
Attendance,0.050786
FormHomeTeam,0.049679
elo,0.047567
elo_away,0.042326
AvgHomeShotsLast5,0.040195
AvgAwayCornersLast5,0.039720
AttackStrengthAway,0.039200
AttackStrengthHome,0.037194
FormAwayTeam,0.036580


In [39]:
(49+40)/160

0.55625

In [91]:
RF_weekly=RF(random_state=30).fit(X_train,y_train)

In [92]:
ypred = RF_weekly.predict(X_test)
y_pred=np.where(ypred>0.5,'correct','wrong')
confusion_RF=confusion_table(y_test,y_pred)

In [93]:
confusion_RF

Truth,correct,wrong
Predicted,,
correct,53,30
wrong,33,44


In [104]:
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
print("Classification Report - Test Set with Best Random Forest")
print(classification_report(y_test, y_pred))

Classification Report - Test Set with Best Random Forest
              precision    recall  f1-score   support

     correct       0.62      0.64      0.63        83
       wrong       0.59      0.57      0.58        77

    accuracy                           0.61       160
   macro avg       0.61      0.60      0.61       160
weighted avg       0.61      0.61      0.61       160



In [46]:
import statsmodels.api as sm
#Logistic regression
glm = sm.GLM(y_train,X_train,family=sm.families.Binomial())
results = glm.fit()

In [47]:
ypred = results.predict(X_test)
y_pred=np.where(ypred>0.5,'correct','wrong')
confusion_logit=confusion_table(y_test,y_pred)

In [48]:
confusion_logit

Truth,correct,wrong
Predicted,,
correct,49,34
wrong,48,29


In [49]:
(49+29)/160

0.4875

In [60]:
clf = DTC(criterion='entropy', random_state=0) 
clf.fit(X_train, y_train)
accuracy_score(y_test, clf.predict(X_test))

0.5375

In [ ]:
ccp_path = clf.cost_complexity_pruning_path(X_train, y_train) kfold = skm.KFold(10,
random_state=1, shuffle=True)